In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/fibe hackathon/dataset/train.csv')

ImportError: ignored

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop('id',axis=1)

In [ ]:
import numpy as np
print(np.any(np.isnan(df)))
print(np.any(np.isinf(df)))

In [ ]:
# dataset contains high extreme or low extreme values
# replacing infinity values with some large value or small values
df.replace([np.inf,-np.inf],-1e9,inplace=True)

In [ ]:
print(np.any(np.isinf(df))) 

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
missing_value_df = missing_value_df.sort_values('percent_missing', ascending=False)
missing_value_df

In [ ]:
mxmiss=missing_value_df[missing_value_df['percent_missing']>=80.0]

In [ ]:
mxmiss.shape

In [ ]:
# so we've 1861 attributes which have more than 80 percent missing values, we need to drop them out
row_index=mxmiss.index
row_index=list(row_index)
df=df.drop(row_index,axis=1)

In [ ]:
df.info()

In [ ]:
# now we've removed  coloumns which have more than 80% missing values

In [ ]:
df.head()

In [ ]:
print(np.any(np.isnan(df))) # true indicates it has nan values

In [ ]:
dff=df.iloc[:,:-1]
dff
y=df.iloc[:,-1]

In [ ]:
dff.head()

In [ ]:
dff.to_csv('data.csv')

In [ ]:
import dask.dataframe as dd 
df_da=dd.read_csv('data.csv') # reading the dataframe as dask_dataframe to make it into chunks and preprocessing ( any process ) will be easy

In [ ]:
def dd(df):
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  return df
df_da=dd(df_da)

In [ ]:
df_da.head()

In [ ]:
# converting into numpy array
def mgen(df):
  data=df.values
  return data
df_dask1=mgen(df_da)

In [ ]:
df_dask1

In [ ]:
# replacing all nan values with zero
df_dask1[np.isnan(df_dask1)]=0

In [ ]:
type(df_dask1)

In [ ]:
def col(df):
  columns=df.columns
  return columns
columns=col(df_da)

In [ ]:
len(columns)

In [ ]:
# standardizing the data ( scaling the data )
from sklearn.preprocessing import MinMaxScaler
def minmax(df_df):
  global scaler
  scaler=MinMaxScaler()
  dd=scaler.fit_transform(df_df)
  return dd
data_scaled=minmax(df_dask1)

In [ ]:
data_scaled

In [ ]:
data_scaled.shape[1]

In [ ]:
"""
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

n_components = (data_scaled.shape[1])
svd = TruncatedSVD(n_components=n_components)
svd.fit_transform(data_scaled) 
explained_var_ratio = svd.explained_variance_ratio_
plt.plot(np.cumsum(explained_var_ratio))
plt.show()
"""

In [ ]:
# as we can observe from the above scree plot the elbow taking place at the range(0-500)
# take n_components as 500

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
data_imputed=svd.fit_transform(data_scaled)

In [ ]:
data_imputed.shape

In [ ]:
svd.components_.shape

In [ ]:
data_imp=np.dot(data_imputed,svd.components_)

In [ ]:
data_imp.shape

In [ ]:
# descaling the data
final_data=scaler.inverse_transform(data_imp)

In [ ]:
final_data

In [ ]:
final_data.shape

In [ ]:
# converting it into a dataframe
final_dataframe=pd.DataFrame(final_data,columns=columns)

In [ ]:
final_dataframe.head()

In [ ]:
import dask.dataframe as dd
ddd=dd.from_pandas(final_dataframe,npartitions=54)

In [ ]:
ddd['target']=y

In [ ]:
import dask.dataframe as dd
ddd=dd.from_pandas(datafr,npartitions=54)

In [ ]:
print(np.any(np.isnan(final_dataframe))) # no nan values

In [ ]:
final_dataframe.isnull().count()

In [ ]:
y

In [ ]:
"""
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

n_components = (final_dataframe.shape[1])
svd = TruncatedSVD(n_components=n_components)
svd.fit_transform(final_dataframe) 
explained_var_ratio = svd.explained_variance_ratio_
plt.plot(np.cumsum(explained_var_ratio))
plt.show()

"""


In [ ]:
"""
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
train_score=[]
test_score=[]
values=[i for i in range(1,20)]
for i in range(1,20):
  model=RandomForestClassifier(n_estimators=100,max_depth=i,max_features='auto',random_state=None,criterion="gini")
  model.fit(x_train,y_train)
  y_pred=model.predict(x_test)
  acc1=accuracy_score(y_pred,y_test)
  y1_pred=model.predict(x_train)
  acc=accuracy_score(y1_pred,y_train) 
  train_score.append(acc)
  test_score.append(acc1) 
plt.plot(values,train_score,'-o',label="train")
plt.plot(values,test_score,'-o',label="test")
plt.xlabel("max_depth")
plt.ylabel("acc")
plt.legend()
plt.show()
"""

In [ ]:
# from the above graph we can conclude that after max_depth= 6 it is getting overfit

In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/fibe hackathon/dataset/test.csv')
df_test.head()

In [ ]:
df_test=df_test.drop('id',axis=1)

In [ ]:
df_test.replace([np.inf,-np.inf],-1e9,inplace=True)

In [ ]:
print(np.any(np.isinf(df_test)))

In [ ]:
import numpy as np

In [ ]:
percent_missing_test1 = df_test.isnull().sum() * 100 / len(df_test)
missing_value_df_test1 = pd.DataFrame({'column_name': df_test.columns,'percent_missing': percent_missing_test1})
missing_value_df_test2= missing_value_df_test1.sort_values('percent_missing', ascending=False)
missing_value_df_test2

In [ ]:
mxmiss_test1=missing_value_df_test1[missing_value_df_test1['percent_missing']>=80.0]

In [ ]:
mxmiss_test1

In [ ]:
mxmiss_test1.shape

In [ ]:
row_index_test=mxmiss_test1.index
row_index_test=list(row_index_test)
df_test=df_test.drop(row_index_test,axis=1) 

In [ ]:
df_test.head()

In [ ]:
print(np.any(np.isinf(df_test)))
print(np.any(np.isnan(df_test)))

In [ ]:
col1=set(final_dataframe.columns)
col2=set(df_test.columns)

In [ ]:
print(len(col1))
print(len(col2))

In [ ]:
col3=col2.intersection(col1)
col4=list(col3)
len(col4)

In [ ]:
for i in final_dataframe.columns:
  if(i in col3):
    continue
  else:
    final_dataframe.drop(i,axis=1)

In [ ]:
x=final_dataframe
Y=y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,Y,random_state=42,test_size=0.33)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=100,max_depth=6,max_features='auto',criterion='entropy')
model.fit(x_train,y_train)

In [ ]:
df_test.to_csv('df_test.csv')

In [ ]:
import dask.dataframe as dd

In [ ]:
df_test_dd=dd.read_csv('df_test.csv')

In [ ]:
df_test_dd.head()

In [ ]:
def dd(df):
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  return df
df_daa=dd(df_test_dd)

In [ ]:
# converting into numpy array
def mgen(df):
  data=df.values
  return data
df_dask2=mgen(df_daa)

In [ ]:
# replacing all nan values with zero
df_dask2[np.isnan(df_dask2)]=0

In [ ]:
type(df_dask2)

In [ ]:
def col(df):
  columns=df.columns
  return columns
columns1=col(df_daa)

In [ ]:
len(columns1)

In [ ]:
# standardizing the data ( scaling the data )
from sklearn.preprocessing import MinMaxScaler
def minmax(df_df):
  global scaler
  scaler=MinMaxScaler()
  dd=scaler.fit_transform(df_df)
  return dd
data_scaled1=minmax(df_dask2)

In [ ]:
data_scaled1

In [ ]:
data_scaled1.shape[1]

In [ ]:
"""
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

n_components = (data_scaled1.shape[1])
svd = TruncatedSVD(n_components=n_components)
svd.fit_transform(data_scaled1) 
explained_var_ratio = svd.explained_variance_ratio_
plt.plot(np.cumsum(explained_var_ratio))
plt.show()
"""

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd1 = TruncatedSVD(n_components=500)
data_imputed1=svd1.fit_transform(data_scaled1)

In [ ]:
data_imputed1.shape

In [ ]:
svd1.components_.shape

In [ ]:
data_imp1=np.dot(data_imputed1,svd1.components_)

In [ ]:
data_imp1.shape

In [ ]:
# descaling the data
final_data1=scaler.inverse_transform(data_imp1)

In [ ]:
final_data1

In [ ]:
# converting it into a dataframe
final_dataframe1=pd.DataFrame(final_data1,columns=columns1)

In [ ]:
final_dataframe1

In [ ]:
y_pred1=model.predict(final_dataframe1)

In [ ]:
columns2=['predicted']

In [ ]:
final_dd=pd.DataFrame(y_pred1.reshape(-1,1),columns=columns2)

In [ ]:
final_dd['predicted'].unique()

In [ ]:
ccc=model.predict_proba(final_dataframe1)
l=[]
for i in range(0,5000):
  l.append(ccc[i][1])

In [ ]:
ccc

In [ ]:
final_dd['predicted_probability']=l

In [ ]:
final_dd.drop(0,axis=1)

In [ ]:
final_dd.to_csv('/content/drive/MyDrive/fibe hackathon/dataset/sub1.csv', index=False)